In [3]:
# Install ViZDoom deps from
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

!apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
!nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
!libopenal-dev timidity libwildmidi-dev unzip ffmpeg

# Boost libraries
!apt-get install libboost-all-dev

# Lua binding dependencies
!apt-get install liblua5.1-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package !nasm
E: Couldn't find any package by glob '!nasm'
E: Unable to locate package !libopenal-dev
E: Couldn't find any package by glob '!libopenal-dev'
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'liblua5.1-0-dev' instead of 'liblua5.1-dev'
The following additional packages will be installed:
  liblua5.1-0 libtool libtool-bin
Suggested packages:
  libtool-doc gcj-jdk
The following NEW packages will be installed:
  liblua5.1-0 liblua5.1-0-dev libtool libtool-bin
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 467 kB of archives.
After this operation,

In [4]:
!pip install sample-factory
!pip install vizdoom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.1/929.1 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cubla

In [5]:
import functools

from sample_factory.algo.utils.context import global_model_factory
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl

from sf_examples.vizdoom.doom.doom_model import make_vizdoom_encoder
from sf_examples.vizdoom.doom.doom_params import add_doom_env_args, doom_override_defaults
from sf_examples.vizdoom.doom.doom_utils import DOOM_ENVS, make_doom_env_from_spec


# Registers all the ViZDoom environments
def register_vizdoom_envs():
    for env_spec in DOOM_ENVS:
        make_env_func = functools.partial(make_doom_env_from_spec, env_spec)
        register_env(env_spec.name, make_env_func)


# Sample Factory allows the registration of a custom Neural Network architecture
# See https://github.com/alex-petrenko/sample-factory/blob/master/sf_examples/vizdoom/doom/doom_model.py for more details
def register_vizdoom_models():
    global_model_factory().register_encoder_factory(make_vizdoom_encoder)


def register_vizdoom_components():
    register_vizdoom_envs()
    register_vizdoom_models()


# parse the command line args and create a config
def parse_vizdoom_cfg(argv=None, evaluation=False):
    parser, _ = parse_sf_args(argv=argv, evaluation=evaluation)
    # parameters specific to Doom envs
    add_doom_env_args(parser)
    # override Doom default values for algo parameters
    doom_override_defaults(parser)
    # second parsing pass yields the final configuration
    final_cfg = parse_full_cfg(parser, argv)
    return final_cfg

In [6]:
## Start the training, this should take around 15 minutes
register_vizdoom_components()

# The scenario we train on today is health gathering
# other scenarios include "doom_basic", "doom_two_colors_easy", "doom_dm", "doom_dwango5", "doom_my_way_home", "doom_deadly_corridor", "doom_defend_the_center", "doom_defend_the_line"
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(
    argv=[f"--env={env}", "--num_workers=8", "--num_envs_per_worker=4", "--train_for_env_steps=4000000"]
)

status = run_rl(cfg)

[2024-07-24 09:33:24,357][00731] register_encoder_factory: <function make_vizdoom_encoder at 0x7decd2c02050>
[2024-07-24 09:33:24,391][00731] Saved parameter configuration for experiment default_experiment not found!
[2024-07-24 09:33:24,392][00731] Starting experiment from scratch!
[2024-07-24 09:33:24,412][00731] Experiment dir /content/train_dir/default_experiment already exists!
[2024-07-24 09:33:24,420][00731] Resuming existing experiment from /content/train_dir/default_experiment...
[2024-07-24 09:33:24,422][00731] Weights and Biases integration disabled
[2024-07-24 09:33:26,964][00731] Queried available GPUs: 0

[2024-07-24 09:33:26,966][00731] Environment var CUDA_VISIBLE_DEVICES is 0

[2024-07-24 09:33:29,394][00731] Automatically setting recurrence to 32
[2024-07-24 09:33:29,397][00731] Starting experiment with the following configuration:
help=False
algo=APPO
env=doom_health_gathering_supreme
experiment=default_experiment
train_dir=/content/train_dir
restart_behavior=resume


In [7]:
from sample_factory.enjoy import enjoy

cfg = parse_vizdoom_cfg(
    argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10"], evaluation=True
)
status = enjoy(cfg)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[2024-07-24 09:52:49,482][00731] Loading existing experiment configuration from /content/train_dir/default_experiment/config.json
[2024-07-24 09:52:49,484][00731] Overriding arg 'num_workers' with value 1 passed from command line
[2024-07-24 09:52:49,487][00731] Adding new argument 'no_render'=True that is not in the saved config file!
[2024-07-24 09:52:49,489][00731] Adding new argument 'save_video'=True that is not in the saved config file!
[2024-07-24 09:52:49,491][00731] Adding new argument 'video_frames'=1000000000.0 that is not in the saved config file!
[2024-07-24 09:52:49,493][00731] Adding new argument 'video_name'=None that is 

In [2]:
from base64 import b64encode
from IPython.display import HTML

mp4 = open("/content/train_dir/default_experiment/replay.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(
    """
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
"""
    % data_url
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train_dir/default_experiment/replay.mp4'

In [1]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [ ]:
from sample_factory.enjoy import enjoy

hf_username = "nithin04"  # insert your HuggingFace username here

cfg = parse_vizdoom_cfg(
    argv=[
        f"--env={env}",
        "--num_workers=1",
        "--save_video",
        "--no_render",
        "--max_num_episodes=10",
        "--max_num_frames=100000",
        "--push_to_hub",
        f"--hf_repository={hf_username}/rl_course_vizdoom_health_gathering_supreme",
    ],
    evaluation=True,
)
status = enjoy(cfg)

ModuleNotFoundError: No module named 'sample_factory'